# To Begin With...

### Name your spark application as `GASPAR_final` or `GROUP_NAME_final`.

<div class='alert alert-info'><b>Any application without a proper name would be promptly killed.</b></div>

In [1]:
%%configure
{"conf": {
    "spark.app.name":"oh-my-git_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5901,application_1589299642358_0390,pyspark,idle,Link,Link,
5906,application_1589299642358_0395,pyspark,busy,Link,Link,
5908,application_1589299642358_0397,pyspark,idle,Link,Link,
5911,application_1589299642358_0400,pyspark,dead,Link,Link,
5915,application_1589299642358_0404,pyspark,idle,Link,Link,
5919,application_1589299642358_0408,pyspark,idle,Link,Link,
5920,application_1589299642358_0409,pyspark,idle,Link,Link,
5921,application_1589299642358_0410,pyspark,idle,Link,Link,
5923,application_1589299642358_0412,pyspark,idle,Link,Link,
5925,application_1589299642358_0414,pyspark,busy,Link,Link,


### Start Spark

In [2]:
# Initialization

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5932,application_1589299642358_0421,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Read the [SBB actual data](https://opentransportdata.swiss/en/dataset/istdaten) in ORC format

In [3]:
sbb = spark.read.orc('/data/sbb/orc/istdaten')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sbb.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- betriebstag: string (nullable = true)
 |-- fahrt_bezeichner: string (nullable = true)
 |-- betreiber_id: string (nullable = true)
 |-- betreiber_abk: string (nullable = true)
 |-- betreiber_name: string (nullable = true)
 |-- produkt_id: string (nullable = true)
 |-- linien_id: string (nullable = true)
 |-- linien_text: string (nullable = true)
 |-- umlauf_id: string (nullable = true)
 |-- verkehrsmittel_text: string (nullable = true)
 |-- zusatzfahrt_tf: string (nullable = true)
 |-- faellt_aus_tf: string (nullable = true)
 |-- bpuic: string (nullable = true)
 |-- haltestellen_name: string (nullable = true)
 |-- ankunftszeit: string (nullable = true)
 |-- an_prognose: string (nullable = true)
 |-- an_prognose_status: string (nullable = true)
 |-- abfahrtszeit: string (nullable = true)
 |-- ab_prognose: string (nullable = true)
 |-- ab_prognose_status: string (nullable = true)
 |-- durchfahrt_tf: string (nullable = true)

### Read the station list data [BFKOORD_GEO](https://opentransportdata.swiss/en/cookbook/hafas-rohdaten-format-hrdf/#Abgrenzung)

In [5]:
metadata = spark.read.csv('/data/sbb/stations/bfkoordgeo.csv', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
metadata.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- StationID: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Remark: string (nullable = true)

In [7]:
metadata.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------+---------+------+----------------+
|StationID|Longitude| Latitude|Height|          Remark|
+---------+---------+---------+------+----------------+
|  0000002|26.074412|44.446770|     0|       Bucuresti|
|  0000003| 1.811446|50.901549|     0|          Calais|
|  0000004| 1.075329|51.284212|     0|      Canterbury|
|  0000005|-3.543547|50.729172|     0|          Exeter|
|  0000007| 9.733756|46.922368|   744|Fideris, Bahnhof|
+---------+---------+---------+------+----------------+
only showing top 5 rows

# Erdem testing

In [45]:
from geopy import distance
import pyspark.sql.types as T
import pyspark.sql.functions  as F
from pyspark.sql.window import Window
import datetime

zurichHB = (47.378177, 8.540192)
cleveland_oh = (49.378177, 9.540192)
print(distance.distance(zurichHB, cleveland_oh).m)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

234402.47499

In [9]:
spark.read.orc("hdfs:///data/sbb/timetables/orc/stop_times").registerTempTable("stop_times_df")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
stop_times_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/stop_times")
stops_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/stops")
trips_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/trips")
calendar_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/calendar")
routes_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/routes")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
trips_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+--------------------+------------------+---------------+------------+
|   route_id|service_id|             trip_id|     trip_headsign|trip_short_name|direction_id|
+-----------+----------+--------------------+------------------+---------------+------------+
|1-1-C-j19-1|  TA+b0001|5.TA.1-1-C-j19-1.3.R|Zofingen, Altachen|            108|           1|
|1-1-C-j19-1|  TA+b0001|7.TA.1-1-C-j19-1.3.R|Zofingen, Altachen|            112|           1|
|1-1-C-j19-1|  TA+b0001|9.TA.1-1-C-j19-1.3.R|Zofingen, Altachen|            116|           1|
|1-1-C-j19-1|  TA+b0001|11.TA.1-1-C-j19-1...|Zofingen, Altachen|            120|           1|
|1-1-C-j19-1|  TA+b0001|13.TA.1-1-C-j19-1...|Zofingen, Altachen|            124|           1|
+-----------+----------+--------------------+------------------+---------------+------------+
only showing top 5 rows

In [12]:
calendar_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------+---------+--------+------+--------+------+
|service_id|monday|tuesday|wednesday|thursday|friday|saturday|sunday|
+----------+------+-------+---------+--------+------+--------+------+
|  TA+b0nx9|  true|   true|     true|    true|  true|   false| false|
|  TA+b03bf|  true|   true|     true|    true|  true|   false| false|
|  TA+b0008|  true|   true|     true|    true|  true|   false| false|
|  TA+b0nxg|  true|   true|     true|    true|  true|   false| false|
|  TA+b08k4|  true|  false|    false|   false| false|   false| false|
+----------+------+-------+---------+--------+------+--------+------+
only showing top 5 rows

In [13]:
routes_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+----------------+---------------+----------+----------+
|   route_id|agency_id|route_short_name|route_long_name|route_desc|route_type|
+-----------+---------+----------------+---------------+----------+----------+
|11-40-j19-1|      801|             040|               |       Bus|       700|
|11-61-j19-1|     7031|             061|               |       Bus|       700|
|11-62-j19-1|     7031|             062|               |       Bus|       700|
|24-64-j19-1|      801|             064|               |       Bus|       700|
|11-83-j19-1|      801|             083|               |       Bus|       700|
+-----------+---------+----------------+---------------+----------+----------+
only showing top 5 rows

In [14]:
@F.udf(T.DoubleType())
def getDistToZurich(lat,lon):
    zurichHB = (47.378177, 8.540192)
    geo = (lat,lon)
    return distance.distance(zurichHB, geo).km
stops_inradius_df= stops_df.withColumn("dist",getDistToZurich("stop_lat","stop_lon")).filter("dist <= 15")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
stops_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+-------------+--------------+
|stop_id|           stop_name|        stop_lat|        stop_lon|location_type|parent_station|
+-------+--------------------+----------------+----------------+-------------+--------------+
|1322000|            Altoggio|46.1672513851495|  8.345807131427|             |              |
|1322001|        Antronapiana| 46.060121674738|8.11361957990831|             |              |
|1322002|              Anzola|45.9898698225697|8.34571729989858|             |              |
|1322003|              Baceno|46.2614983591677|8.31925293162473|             |              |
|1322004|Beura Cardezza, c...|46.0790618438814|8.29927439970313|             |              |
+-------+--------------------+----------------+----------------+-------------+--------------+
only showing top 5 rows

In [16]:
stops_df.withColumn("dist",getDistToZurich("stop_lat","stop_lon")).count()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

30631

In [17]:
stops_df.withColumn("dist",getDistToZurich("stop_lat","stop_lon")).filter("dist <= 15").count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1880

In [18]:
stops_inradius_df= stops_df.withColumn("dist",getDistToZurich("stop_lat","stop_lon")).filter("dist <= 15")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
stops_inradius_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+----------------+----------------+-------------+--------------+------------------+
|    stop_id|           stop_name|        stop_lat|        stop_lon|location_type|parent_station|              dist|
+-----------+--------------------+----------------+----------------+-------------+--------------+------------------+
|    8500926|Oetwil a.d.L., Sc...|47.4236270123012| 8.4031825286317|             |              |11.510621170348637|
|    8502186|Dietikon Stoffelbach|47.3934058321612|8.39894248049007|             |      8502186P|10.798854022201352|
|8502186:0:1|Dietikon Stoffelbach|47.3934666445388|8.39894248049007|             |      8502186P|10.799910068148662|
|8502186:0:2|Dietikon Stoffelbach|47.3935274568464|8.39894248049007|             |      8502186P|10.800970241796682|
|   8502186P|Dietikon Stoffelbach|47.3934058321612|8.39894248049007|            1|              |10.798854022201352|
+-----------+--------------------+----------------+-------------

In [20]:

stop_times_df = stop_times_df.drop("pickup_type","drop_off_type")
stop_times_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-----------+-------------+
|             trip_id|arrival_time|departure_time|    stop_id|stop_sequence|
+--------------------+------------+--------------+-----------+-------------+
|1.TA.1-1-B-j19-1.1.R|    04:20:00|      04:20:00|8500010:0:3|            1|
|1.TA.1-1-B-j19-1.1.R|    04:24:00|      04:24:00|8500020:0:3|            2|
|1.TA.1-1-B-j19-1.1.R|    04:28:00|      04:28:00|8500021:0:5|            3|
|1.TA.1-1-B-j19-1.1.R|    04:30:00|      04:30:00|8517131:0:2|            4|
|1.TA.1-1-B-j19-1.1.R|    04:32:00|      04:32:00|8500300:0:5|            5|
+--------------------+------------+--------------+-----------+-------------+
only showing top 5 rows

# Get stops in radius of 15km

In [21]:
stops_inradius_df = stops_inradius_df.drop("dist")
stops_joined_df = stop_times_df.join(stops_inradius_df,stop_times_df.stop_id == stops_inradius_df.stop_id ).drop(stops_inradius_df.stop_id)
stops_joined_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+---------------+-------------+------------------+----------------+----------------+-------------+--------------+
|             trip_id|arrival_time|departure_time|        stop_id|stop_sequence|         stop_name|        stop_lat|        stop_lon|location_type|parent_station|
+--------------------+------------+--------------+---------------+-------------+------------------+----------------+----------------+-------------+--------------+
|5.TA.1-1-A-j19-1.3.H|    02:42:00|      02:42:00|    8503305:0:3|            2|        Effretikon|  47.42593043029|8.68666388288076|             |      8503305P|
|5.TA.1-1-A-j19-1.3.H|    02:46:00|      02:46:00|    8503306:0:3|            3|         Dietlikon|47.4203145327083|8.61925430395105|             |      8503306P|
|5.TA.1-1-A-j19-1.3.H|    02:50:00|      02:50:00|    8503147:0:1|            4|         Stettbach| 47.397334167601|8.59613166853459|             |      8503147P|
|5.TA.1-1-A-j19-1.3.H|

In [22]:
stops_inradius_df.filter("stop_id == '8503306P' ").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+----------------+----------------+-------------+--------------+
| stop_id|stop_name|        stop_lat|        stop_lon|location_type|parent_station|
+--------+---------+----------------+----------------+-------------+--------------+
|8503306P|Dietlikon|47.4201929698596|8.61925430395105|            1|              |
+--------+---------+----------------+----------------+-------------+--------------+

# Create stop connections 

In [23]:
w1 = Window.partitionBy("trip_id").orderBy("arrival_time")
#Previous stop name
df_prev_stop = stops_joined_df.withColumn('prev_stop_name',
                        F.lag(stops_joined_df.stop_name)
                                 .over(w1))
#Previous stop id
df_prev_stop_id = df_prev_stop.withColumn('prev_stop_id',
                        F.lag(df_prev_stop.stop_id)
                                 .over(w1))
#Departure time from previous stop                            
df_prev_departure = df_prev_stop_id.withColumn('prev_departure',
                        F.lag(df_prev_stop_id.departure_time)
                                 .over(w1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df_trip_network  = df_prev_departure.select("trip_id","prev_departure","arrival_time","prev_stop_name","stop_name","prev_stop_id","stop_id")
df_trip_network.show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+------------+--------------------+--------------------+------------+------------+
|             trip_id|prev_departure|arrival_time|      prev_stop_name|           stop_name|prev_stop_id|     stop_id|
+--------------------+--------------+------------+--------------------+--------------------+------------+------------+
|1.TA.26-18-j19-1.1.H|          null|    28:45:00|                null|      Zürich, Rehalp|        null|     8591315|
|1.TA.26-18-j19-1.1.H|      28:45:00|    28:46:00|      Zürich, Rehalp|Zürich, Friedhof ...|     8591315|     8591142|
|1.TA.26-18-j19-1.1.H|      28:46:00|    28:47:00|Zürich, Friedhof ...|     Zürich Balgrist|     8591142|     8530811|
|1.TA.26-18-j19-1.1.H|      28:47:00|    28:48:00|     Zürich Balgrist|    Zürich, Burgwies|     8530811|     8591104|
|1.TA.26-18-j19-1.1.H|      28:48:00|    28:50:00|    Zürich, Burgwies|Zürich, Wetlistrasse|     8591104|     8591429|
|1.TA.26-18-j19-1.1.H|      28:50:00|    28:50:0

In [25]:
# Filter reasonable hours between 07-21
df_trip_network_reasonable = df_trip_network.filter(" CAST(SUBSTRING(arrival_time,1,2) AS int) BETWEEN 7 AND 21 ")
df_trip_network_reasonable.show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+------------+--------------------+--------------------+------------+------------+
|             trip_id|prev_departure|arrival_time|      prev_stop_name|           stop_name|prev_stop_id|     stop_id|
+--------------------+--------------+------------+--------------------+--------------------+------------+------------+
|1.TA.63-138-Y-j19...|          null|    18:22:00|                null|           Zürich HB|        null|8503000:0:16|
|10.TA.80-55-Y-j19...|          null|    07:38:00|                null|             Thalwil|        null| 8503202:0:5|
|10.TA.80-55-Y-j19...|      07:39:00|    07:48:00|             Thalwil|           Zürich HB| 8503202:0:5|8503000:0:10|
|1005.TA.26-131-j1...|          null|    16:14:00|                null|     Horgen, Bahnhof|        null| 8503855:0:F|
|1005.TA.26-131-j1...|      16:14:00|    16:15:00|     Horgen, Bahnhof|Horgen, Gumelenst...| 8503855:0:F|     8589111|
|1005.TA.26-131-j1...|      16:15:00|    16:16:0

In [26]:
stops_inradius_df.alias("stops1").crossJoin(stops_inradius_df.alias("stops2")).show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+---------------+-------------+--------------+-----------+--------------------+----------------+----------------+-------------+--------------+
|stop_id|           stop_name|        stop_lat|       stop_lon|location_type|parent_station|    stop_id|           stop_name|        stop_lat|        stop_lon|location_type|parent_station|
+-------+--------------------+----------------+---------------+-------------+--------------+-----------+--------------------+----------------+----------------+-------------+--------------+
|8500926|Oetwil a.d.L., Sc...|47.4236270123012|8.4031825286317|             |              |    8500926|Oetwil a.d.L., Sc...|47.4236270123012| 8.4031825286317|             |              |
|8500926|Oetwil a.d.L., Sc...|47.4236270123012|8.4031825286317|             |              |    8502186|Dietikon Stoffelbach|47.3934058321612|8.39894248049007|             |      8502186P|
|8500926|Oetwil a.d.L., Sc...|47.4236270123012|8.403182

# Create Walking Connections

In [27]:
@F.udf(T.DoubleType())
def getDist(lat1,lon1,lat2,lon2):
    geo1 = (lat1,lon1)
    geo2 = (lat2,lon2)
    dist = distance.distance(geo1, geo2).m
    return dist

stops_inradius_df = stops_inradius_df.drop("dist")

stopDist = stops_inradius_df.alias("stops1").crossJoin(stops_inradius_df.alias("stops2"))\
.where(getDist("stops1.stop_lat","stops1.stop_lon","stops2.stop_lat","stops2.stop_lon")<= 500)\
.withColumn("dist",getDist("stops1.stop_lat","stops1.stop_lon","stops2.stop_lat","stops2.stop_lon"))


#Other option is this but don't know if it is way more efficient or more or less similiar
#stopDistFiltered = stops_inradius_df.alias("stops1").join(stops_inradius_df.alias("stops2")\
#,getDist("stops1.stop_lat","stops1.stop_lon","stops2.stop_lat","stops2.stop_lon")<=500)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
stopDistFiltered

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'stopDistFiltered' is not defined
Traceback (most recent call last):
NameError: name 'stopDistFiltered' is not defined



In [29]:


stopDist = stopDist.select(F.col("stops1.stop_id").alias("stop1_id")\
               ,F.col("stops1.stop_name").alias("stop1_name")\
               ,F.col("stops1.stop_lat").alias("stop1_lat")\
               ,F.col("stops1.stop_lon").alias("stop1_lon")\
               ,F.col("stops2.stop_id").alias("stop2_id")\
               ,F.col("stops2.stop_name").alias("stop2_name")\
               ,F.col("stops2.stop_lat").alias("stop2_lat")\
               ,F.col("stops2.stop_lon").alias("stop2_lon")\
               ,F.col("dist"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
#stopDist.write.parquet("user/boecuego/walk_stops.parquet")

walk_stops = spark.read.parquet("user/boecuego/walk_stops.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
walk_stops.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14414

In [32]:

walk_stops = walk_stops.drop("stop1_lat","stop1_lon","stop2_lat","stop2_lon").filter("dist > 0")
walk_stops.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+-----------+--------------------+------------------+
|   stop1_id|          stop1_name|   stop2_id|          stop2_name|              dist|
+-----------+--------------------+-----------+--------------------+------------------+
|    8500926|Oetwil a.d.L., Sc...|    8590616|Geroldswil, Schwe...|122.61607002692374|
|    8500926|Oetwil a.d.L., Sc...|    8590737|Oetwil an der Lim...|  300.671189772556|
|    8502186|Dietikon Stoffelbach|8502186:0:1|Dietikon Stoffelbach| 6.761029676373361|
|    8502186|Dietikon Stoffelbach|8502186:0:2|Dietikon Stoffelbach|  13.5220516429909|
|    8502186|Dietikon Stoffelbach|    8502270|         Bergfrieden| 478.7847706261187|
|    8502186|Dietikon Stoffelbach|8502270:0:1|         Bergfrieden|485.54354073977174|
|    8502186|Dietikon Stoffelbach|    8590200|Bergdietikon, Bal...|483.67990364227626|
|    8502186|Dietikon Stoffelbach|    8590203| Bergdietikon, Dünni|455.34915607314645|
|8502186:0:1|Dietikon Stoffelbach|    85021

# Create walking connections for each hour respect to timetable from connections

In [33]:
df_trip_network_reasonable.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+------------+--------------------+--------------------+------------+------------+
|             trip_id|prev_departure|arrival_time|      prev_stop_name|           stop_name|prev_stop_id|     stop_id|
+--------------------+--------------+------------+--------------------+--------------------+------------+------------+
|1.TA.63-138-Y-j19...|          null|    18:22:00|                null|           Zürich HB|        null|8503000:0:16|
|10.TA.80-55-Y-j19...|          null|    07:38:00|                null|             Thalwil|        null| 8503202:0:5|
|10.TA.80-55-Y-j19...|      07:39:00|    07:48:00|             Thalwil|           Zürich HB| 8503202:0:5|8503000:0:10|
|1005.TA.26-131-j1...|          null|    16:14:00|                null|     Horgen, Bahnhof|        null| 8503855:0:F|
|1005.TA.26-131-j1...|      16:14:00|    16:15:00|     Horgen, Bahnhof|Horgen, Gumelenst...| 8503855:0:F|     8589111|
|1005.TA.26-131-j1...|      16:15:00|    16:16:0

In [34]:
df_trip_network_reasonable.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trip_id: string (nullable = true)
 |-- prev_departure: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- prev_stop_name: string (nullable = true)
 |-- stop_name: string (nullable = true)
 |-- prev_stop_id: string (nullable = true)
 |-- stop_id: string (nullable = true)

In [35]:
df_trip_network_reasonable.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1782051

In [36]:
df_stop_hours = df_trip_network_reasonable.select("arrival_time","stop_id")

df_walk_stops_whours = df_stop_hours.join(walk_stops,df_stop_hours.stop_id == walk_stops.stop1_id).drop("stop_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
columns = df_walk_stops_whours.columns[1:]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df_walk_stops_whours  = df_walk_stops_whours.select(F.col("arrival_time").alias("departure_time"),*columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
df_walk_stops_whours.sample(0.1,seed=12345).show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------+--------------------+--------+--------------------+------------------+
|departure_time|stop1_id|          stop1_name|stop2_id|          stop2_name|              dist|
+--------------+--------+--------------------+--------+--------------------+------------------+
|      18:51:00| 8578679|   Würenlos, Bettlen| 8590314|Würenlos, Büntens...|276.84035282176313|
|      07:51:00| 8578679|   Würenlos, Bettlen| 8590314|Würenlos, Büntens...|276.84035282176313|
|      10:05:00| 8590314|Würenlos, Büntens...| 8578679|   Würenlos, Bettlen|276.84035282176313|
|      10:35:00| 8590314|Würenlos, Büntens...| 8578679|   Würenlos, Bettlen|276.84035282176313|
|      10:36:00| 8578679|   Würenlos, Bettlen| 8590314|Würenlos, Büntens...|276.84035282176313|
|      12:36:00| 8578679|   Würenlos, Bettlen| 8590314|Würenlos, Büntens...|276.84035282176313|
|      14:05:00| 8590314|Würenlos, Büntens...| 8578679|   Würenlos, Bettlen|276.84035282176313|
|      14:21:00| 8578679|   Würenlos, Be

In [40]:
df_walk_stops_whours.select("stop1_name").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1491

In [41]:
df_walk_stops_whours.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

13494097

In [54]:
# Adding the walking_duration_min column considering that pedestrian walks 50m/min
df_walk_stops_whours = df_walk_stops_whours.withColumn("walking_duration_min", F.ceil(F.col("dist")/50))
df_walk_stops_whours.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------+--------------------+--------+--------------------+------------------+--------------------+
|departure_time|stop1_id|          stop1_name|stop2_id|          stop2_name|              dist|walking_duration_min|
+--------------+--------+--------------------+--------+--------------------+------------------+--------------------+
|      20:08:00| 8578679|   Würenlos, Bettlen| 8590314|Würenlos, Büntens...|276.84035282176313|                   6|
|      20:08:00| 8590314|Würenlos, Büntens...| 8578679|   Würenlos, Bettlen|276.84035282176313|                   6|
|      20:38:00| 8578679|   Würenlos, Bettlen| 8590314|Würenlos, Büntens...|276.84035282176313|                   6|
|      20:38:00| 8590314|Würenlos, Büntens...| 8578679|   Würenlos, Bettlen|276.84035282176313|                   6|
|      21:08:00| 8578679|   Würenlos, Bettlen| 8590314|Würenlos, Büntens...|276.84035282176313|                   6|
|      21:08:00| 8590314|Würenlos, Büntens...| 8578679|   Würenl

In [56]:
df_walk_stops_whours = df_walk_stops_whours.withColumn("arrival_time", F.from_unixtime(F.unix_timestamp(F.col("departure_time").cast("timestamp"))
                                                                 + F.col("walking_duration_min")*60, "HH:mm:ss"))
df_walk_stops_whours.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------+--------------------+--------+--------------------+------------------+--------------------+------------+
|departure_time|stop1_id|          stop1_name|stop2_id|          stop2_name|              dist|walking_duration_min|arrival_time|
+--------------+--------+--------------------+--------+--------------------+------------------+--------------------+------------+
|      20:08:00| 8578679|   Würenlos, Bettlen| 8590314|Würenlos, Büntens...|276.84035282176313|                   6|    20:14:00|
|      20:08:00| 8590314|Würenlos, Büntens...| 8578679|   Würenlos, Bettlen|276.84035282176313|                   6|    20:14:00|
|      20:38:00| 8578679|   Würenlos, Bettlen| 8590314|Würenlos, Büntens...|276.84035282176313|                   6|    20:44:00|
|      20:38:00| 8590314|Würenlos, Büntens...| 8578679|   Würenlos, Bettlen|276.84035282176313|                   6|    20:44:00|
|      21:08:00| 8578679|   Würenlos, Bettlen| 8590314|Würenlos, Büntens...|276.8403528217